# New Friday work

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [53]:
# read product Price Data
df = pd.read_csv('PriceLog.csv', skiprows=2)

In [54]:
# Clean Data
df = df.drop(['Unnamed: 0','IsMovedtoProduction', 'ID', 'SyncedDate', 'Description'], axis=1)
df['CreatedDate'] = pd.to_datetime(df['CreatedDate']).dt.date
df = df[df['CreatedDate']>pd.to_datetime('2021-09-28')]
df['Store'] = df['StoreID'].apply(lambda x: f'Store_{x}')

In [55]:
mapping_info = {
    'MacBook Pro 13inch With Touch Bar': [7062439, 262527, 2913093,2404805,301946,253607,1708049,257436],
    'Apple macbook pro 16 inch': [2731965,2321956,2397169,257442,2731965],
    'iPhone 13 Pro 128GB Silver':[8584909,8601848,8596835,8584909,8305247,8491089,8328367],
    'iPhone 13 mini 256GB Starlight':[8305347,8305899,8328379,8489928],
    'Sony PlayStation 5 Console Disc Version':[1701305,314936,2728877,8231121,2729060,254530],
    'Sony PlayStation 5 Digital Edition Console':[3753028,7308958,3334223,3753028],
    'Dyson V11 Absolute Cordless Vacuum Cleaner Blue':[2756958,242455,2307555,319630],
}

In [56]:
df_sales_price = df.pivot_table('SalesPrice', ['ProductID', 'Store'], 'CreatedDate')
df_sales_price = df_sales_price.fillna(method='ffill', axis=1)
df_sales_price = df_sales_price.fillna(method='backfill', axis=1)

df_list_price = df.pivot_table('ListPrice', ['ProductID', 'Store'], 'CreatedDate')
df_list_price = df_list_price.fillna(method='ffill', axis=1)
df_list_price = df_list_price.fillna(method='backfill', axis=1)

In [57]:
def make_dfs(mapping, df):
    dfs = {}
    for key, value in mapping.items():
        new_df = df[df.index.get_level_values(0).isin(value)]
        new_df.index = new_df.index.get_level_values(1)
        dfs[key] = new_df.T
    
    return dfs

In [58]:
all_sales_dfs = make_dfs(mapping_info, df_sales_price)
all_list_dfs = make_dfs(mapping_info, df_list_price)

In [62]:
def show_graph(product):
    df = all_sales_dfs[product].reset_index()
    fig = px.line(df, x='CreatedDate', y=df.columns, markers=True,
                  title=product)
    fig.add_vrect(
            x0="2021-11-18", x1="2021-11-29",
            fillcolor="LightSalmon", opacity=0.5,
            layer="below", line_width=0,
        )

    fig.show()

In [63]:
def show_graph_with_list(product):

    sony_col = all_sales_dfs[product].reset_index()
    sony_col.columns = [f'{col}_DiscountedPrice' if col!='CreatedDate' else col for col in sony_col.columns]
    sony_col_list = all_list_dfs[product].reset_index()
    sony_col_list.columns = [f'{col}_FullPrice' if col!='CreatedDate' else col for col in sony_col_list.columns]
    sony = pd.merge(sony_col, sony_col_list, on='CreatedDate')
    fig = px.line(sony, x='CreatedDate', y=sorted(sony.columns), markers=True, 
              title=product)
    fig.add_vrect(
            x0="2021-11-18", x1="2021-11-29",
            fillcolor="LightSalmon", opacity=0.5,
            layer="below", line_width=0,
        )

    fig.show()

# Iphone 13 Pro 128GB Silver 

In [64]:
show_graph_with_list('iPhone 13 Pro 128GB Silver')

# Sony PS5 Console

In [65]:
show_graph_with_list('Sony PlayStation 5 Console Disc Version')


# Dyson V11 Absolute Cordless Vacuum Cleaner Blue

In [66]:
show_graph_with_list('Dyson V11 Absolute Cordless Vacuum Cleaner Blue')

# MacBook Pro 13inch With Touch Bar

In [67]:
show_graph_with_list('MacBook Pro 13inch With Touch Bar')